In [40]:
# https://pypi.org/project/gtfs-functions
# NOTE 10/21/2024 - needed to update some func calls in gtfs_functions to call on v.4 of h3 module (was still using old v3 calls)

# GTFS directory - I:\Projects\Darren\PPA3_GIS\ConveyalLayers\GTFS

# NOTE - you also need to switch to gtfs-funcs env
import geopandas as gpd
import datetime as dt
from pathlib import Path

import pandas as pd

from gtfs_functions import Feed

gtfs_path = r"I:\Projects\Darren\PPA3_GIS\ConveyalLayers\GTFS\sacrt.zip"
opname = Path(gtfs_path).stem

start_date='2024-01-08'
end_date='2024-01-12'
feed = Feed(gtfs_path, start_date=start_date, end_date=end_date)

# output GIS data path
gpkg_path = r'I:\Projects\Darren\HiFrequencyTransit\hifreq_gtfs.gpkg'


In [39]:
# import inspect
# inspect.signature(Feed)

feed.calendar


,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,1,1,1,1,1,1,0,0,20240107,20240406
1,6,0,0,0,0,0,1,0,20240107,20240406
2,7,0,0,0,0,0,0,1,20240107,20240406


In [41]:
#  frequency of departures for each route
# potentially could make a good supplemental layer of hf points based on where lines intersect

line_freq = feed.lines_freq # if throws ValueError: 'nan' is not in list, make sure your time_bounds cover full 24hr day
line_freq.head(2)

INFO:root:Reading "stop_times.txt".
INFO:root:get trips in stop_times
INFO:root:accessing trips
INFO:root:Reading "routes.txt".
INFO:root:Reading "trips.txt".
INFO:root:Reading "calendar.txt".
INFO:root:Reading "calendar_dates.txt".
INFO:root:The busiest date/s of this feed or your selected date range is/are:  ['2024-01-08', '2024-01-09', '2024-01-10', '2024-01-11', '2024-01-12'] with 2550 trips.
INFO:root:In the case that more than one busiest date was found, the first one will be considered.
INFO:root:In this case is 2024-01-08.
INFO:root:Reading "stop_times.txt".
INFO:root:_trips is defined in stop_times
INFO:root:Reading "stops.txt".
INFO:root:computing patterns
INFO:root:Reading "shapes.txt".


,route_id,route_name,direction_id,window,min_per_trip,ntrips,geometry
0,001,1 GREENBACK,0,0:00-6:00,120,3,"LINESTRING (-121.26715 38.67928, -121.26715 38..."
1,001,1 GREENBACK,0,15:00-19:00,15,16,"LINESTRING (-121.26715 38.67928, -121.26715 38..."


In [42]:
# frequency of departures for each segment
segments_freq = feed.segments_freq
seg_hifreq = segments_freq.loc[segments_freq['min_per_trip'] <= 20]

INFO:root:Getting segments...
INFO:root:Projecting stops onto shape...
INFO:root:Interpolating stops onto shape...
INFO:root:Sorting shape points and stops...
INFO:root:segments_df: 5050, geometry: 5050
INFO:root:adding data for all lines.


In [43]:
# start with tbl where each record is seg with start and end stop
# want to convert into single list where each record is a stop.
# to do so, take the end stop IDs and append to start stop IDs
seg_spec_cols = ['geometry' , 'segment_id', 'segment_name']
cols_starts = ['end_stop_name', 'end_stop_id', *seg_spec_cols]
cols_ends = ['start_stop_name', 'start_stop_id', *seg_spec_cols]
rename = {'start_stop_id': 'stop_id', 'start_stop_name':'stop_name', 'end_stop_id': 'stop_id', 'end_stop_name':'stop_name'}

hf_starts = seg_hifreq[[f for f in seg_hifreq if f not in cols_starts]].rename(columns=rename)
hf_ends = seg_hifreq[[f for f in seg_hifreq if f not in cols_ends]].rename(columns=rename)
hf_combd = pd.concat([hf_starts, hf_ends]).drop_duplicates()
hf_combd = hf_combd.loc[(hf_combd['window'].isin(['6:00-9:00', '15:00-19:00'])) \
                    & (hf_combd['route_id'] != 'ALL_LINES')] # only want to consider frequencies during AM/PM peak



# then, need to only get stops where both AM *and* PM peak meets frequency threshold 
gb_ampmpk = ['route_id', 'direction_id', 'stop_id']
windcnt = hf_combd.groupby(gb_ampmpk)['window'].count().reset_index()
hf_combd2 = hf_combd.merge(windcnt, how='left', on=['route_id', 'direction_id', 'stop_id'], suffixes=('', '_cnt'))
hf_combd2 = hf_combd2.loc[hf_combd2['window_cnt'] > 1]


# then get stop IDs where there are 2+ different route IDs serving it.

# ISSUE 10/21/2024 - This is *very* conservative because it only counts if the *exact* stop has 2 or more routes with high-freq.
# In real world, need to look at multiple stops (e.g., at an intersection, you have different stop IDs for N-S vs. E-W roads; you need to
# count all stops at an intersection if the stops for both directions have hi-freq
# maybe this needs to be semi-manual process?
hfcombd3 = hf_combd2[['route_id', 'route_name', 'direction_id', 'stop_id']].drop_duplicates()
hfr_stop = hfcombd3.groupby(['stop_id'])['route_id'].count().reset_index()
display(hfr_stop.sort_values(by='route_id', ascending=False).head())

# convert to geodataframe
hfcombd3 = hfcombd3.merge(feed.stops, on='stop_id', how='left')
hfcombd3 = gpd.GeoDataFrame(hfcombd3, geometry='geometry')
# hfcombd3.head(2)
hfcombd3.loc[hfcombd3['stop_id'] == '1184']

,stop_id,route_id
1,1184,3
287,9812,2
219,7033,2
185,658,2
186,668,2


,route_id,route_name,direction_id,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,geometry
0,001,1 GREENBACK,0,1184,1184,ARCADIA DR & GREENBACK LN (SB),Buses head SB,38.679128,-121.267253,0,POINT (-121.26725 38.67913)
293,001,1 GREENBACK,1,1184,1184,ARCADIA DR & GREENBACK LN (SB),Buses head SB,38.679128,-121.267253,0,POINT (-121.26725 38.67913)
295,021,21 SUNRISE,1,1184,1184,ARCADIA DR & GREENBACK LN (SB),Buses head SB,38.679128,-121.267253,0,POINT (-121.26725 38.67913)


In [44]:
hf_combd2.loc[hf_combd2['route_id'] == '021']

,route_id,route_name,direction_id,stop_name,window,min_per_trip,ntrips,stop_id,window_cnt
131,021,21 SUNRISE,1,SUNRISE BLVD & GREENBACK LN (NB),15:00-19:00,15,16,2807,2
132,021,21 SUNRISE,1,SUNRISE BLVD & GREENBACK LN (NB),6:00-9:00,15,12,2807,2
697,021,21 SUNRISE,1,ARCADIA DR & GREENBACK LN (SB),15:00-19:00,15,16,1184,2
698,021,21 SUNRISE,1,ARCADIA DR & GREENBACK LN (SB),6:00-9:00,15,12,1184,2


In [53]:
# 10/21/2024 - debugging to see why gtfs tool is giving too-short headdways for route 21, which only runs ~30min headways

cols = ['route_id', 'stop_id', 'arrival_time', 'service_id']

stimes = feed.stop_times
tdf = stimes.loc[(stimes['route_id'] == '021') & (stimes['stop_id'] == '2807')]
tdf.sort_values(by='arrival_time')[cols]

tdf.arrival_time.values[0]
tdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 66 entries, 12613 to 14120
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   trip_id         66 non-null     object  
 1   route_id        66 non-null     object  
 2   pattern_id      66 non-null     object  
 3   route_pattern   66 non-null     object  
 4   pattern_name    66 non-null     object  
 5   route_name      66 non-null     object  
 6   service_id      66 non-null     int64   
 7   direction_id    66 non-null     int64   
 8   shape_id        66 non-null     object  
 9   arrival_time    66 non-null     float64 
 10  departure_time  66 non-null     float64 
 11  stop_id         66 non-null     object  
 12  stop_sequence   66 non-null     int64   
 13  pickup_type     66 non-null     int64   
 14  drop_off_type   66 non-null     int64   
 15  stop_code       66 non-null     int64   
 16  stop_name       66 non-null     object  
 17  stop_des

In [31]:
# export to GIS feature class (remember, cannot use arcpy because you are in separate gtfs-funcs environment)
hfcombd3.to_file(gpkg_path, driver='GPKG', layer=f'hifreq_{opname}')
